# Simulating drainage in soils
The movement of water throughout the soil profile is complex due to the great number of physical interactions between the various minerals, particle sizes, pores, water, organic matter and dissolved salts. Various formulations have been proposed to simulate this movement.

The drainage function used in the crop model AquaCrop is described in the Aquacrop Reference Manual, Chapter 3 - Calculation Procedures. It is a empyrical and realistic although somewhat simple procedure, which we try to explain here, whith implementations in the Python language.

These calculations are not yet implemented in AgmetPy because it would limit the flexibility to calculate the soil water balance in different timesteps (i.e. other than a day). So we look for a more general approach.

## The Acracrop drainage function

The drainage function is defined as

$
\frac{d\theta}{dt} = \tau (\theta_{sat} - \theta_{fc}) \frac{e ^ {(\theta - \theta_{fc})} - 1}{e ^ {(\theta_{sat} - \theta_{fc})} - 1}
$

where:
*   $\theta_{sat}$ volumetric soil moisture at saturation
*   $\theta$ volumetric soil moisture
*   $\theta_{fc}$ volumetric soil moisture at field capacity
*   $\tau$ drainage characteristic.

The drainage characteristic $\tau$ is calculated as a function of saturated hydraulic conductivity ($K_{sat}$), as follows

$
0 \leq \tau = 0.0866 K_{sat} ^ {0.35} \leq 1
$

where $K_{sat}$ is the saturated hydraulic conductivity of the soil compartment in $\text{mm day}^\text{-1}$.

For this purpose we will use numpy.

In [19]:
import numpy as np

Lets create the Pythonic version of both equations above.

In [21]:
def _tau(ksat):
    '''
    Calculates the drainage characteristic.

    Parameters
    ----------
    ksat: float, array_like
        saturated hydraulic conductivity [mm/day]

    '''
    return np.clip(0.0866 * ksat ** 0.35, 0, 1)

def drainage_function(theta, theta_fc, theta_sat, ksat):
    '''
    Calculates the drainage ability for a soil layer.

    Parameters
    ----------
    theta: float, array_like
    soil volumetric moisture content.

    theta_fc: float, array_like
    soil volumetric moisture content at saturation.

    theta_sat: float, array_like
    soil volumetric moisture content at saturation.

    ksat:
    saturated hydraulic conductivity
    '''
    tau = _tau(ksat)
    drainage_ability = tau * (theta_sat - theta_fc) * (np.exp(theta - theta_fc) - 1) / (np.exp(theta_sat - theta_fc) - 1)
    drainage_ability = np.clip(drainage_ability, 0, np.maximum(theta - theta_fc, 0))
    return drainage_ability

Lets see an example with $\theta = 0.22$, $\theta_{fc} = 0.180$, $\theta_{sat} = 0.22$ and $K_{sat} = 400$. You may change the values as you want.

In [25]:
# example
drainage_function(0.22, 0.180, 0.22, 400)

0.028203232024876745

In [24]:
theta = np.array([
    0.22, 0.21, 0.20, 0.19, 0.18, 0.17, 0.16, 0.15, 0.14, 0.13
])

theta_fc = np.array([
    0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18
])

theta_sat = np.array([
    0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22
])

ksat = np.array([
    400, 400, 400, 400, 400, 400, 400, 400, 400, 400
])

drainage_function(theta, theta_fc, theta_sat, ksat)

array([0.02820323, 0.02104631, 0.0139606 , 0.0069454 , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [ ]:
def moisture_needed(drainage_ability, theta_fc, theta_sat, ksat):
    '''
    Calculates the volumetric moisture necessary for a soil to have a given drainage ability.

    Parameters
    ----------
    theta: float, array_like
    soil volumetric moisture content.

    theta_fc: float, array_like
    soil volumetric moisture content at saturation.

    theta_sat: float, array_like
    soil volumetric moisture content at saturation.

    ksat:
    saturated hydraulic conductivity

    '''
    tau = _tau(ksat)
    theta = np.log(1 + drainage_ability * (np.exp(theta_sat - theta_fc) - 1) / (tau * (theta_sat - theta_fc))) + theta_fc
    exc = np.maximum(theta - theta_sat, 0)
    theta = np.minimum(theta, theta_sat)
    return theta, exc

From Aquacrop Reference Manual, Chapter 3 - Calculation procedures:
>[...] Subsequently the soil water content of the top compartment is updated. The same calculations are repeated for the successive compartments. It is thereby assumed that the cumulative drainage amount $\sum{D_i} = D_1 + D_2 + \dots + D_n$ will pass through any compartment as long as its drainage ability is greater than or equal to the drainage ability of the upperlying compartment. By comparing drainage abilities and not soil water contents, the calculation procedure is independent of the soil layer to which succeeding compartments may belong.

>If a compartment is reached which drainage ability is smaller than the upperlying compartment, $\sum{D_i}$ will be stored in that compartment, thereby increasing its soil water content and its drainage ability (Fig 3.7e). If the soil water content of the compartment becomes thereby as high that its drainage ability becomes equal to the drainage ability of the upperlying compartment, the excess of the cumulative drainage amount, increased with the calculated drainage amount $D_i$ of that compartment, will be transferred to the underlying compartment (as is the case in compartment 4 and 5 of Figure 3.7e). If the entire cumulative drainage amount can be stored in a compartment without increasing its soil water content in such a way that its drainage ability becomes equal to that of the upperlying compartment (as is the case in compartment 6), only the calculated drainage amount of that compartment will be transferred to the underlying compartment. If in a compartment the soil water content remains below field capacity, its drainability is zero and no water is transferred to the underlying compartment. At the bottom of the soil profile, the remaining part of $\sum{D}$ will be lost as deep percolation ($\sum D =  DP$).
[...]